* Image denoising is to remove noise from a noisy image, so as to restore the true image
* CIFAR10 dataset
* Image is grayscale of size 48*48

# Importing libraries

In [ ]:
from keras.datasets import fashion_mnist, mnist
import numpy as np
import matplotlib.pyplot as plt

from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Dropout
from keras.models import Model

import os,cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from pylab import rcParams
rcParams['figure.figsize'] = 20, 10

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import pandas as pd# Any results you write to the current directory are saved as output.
from IPython.display import display, Image

from keras.preprocessing.image import load_img
from keras import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.models import load_model
from keras.optimizers import Adam
from keras.utils.vis_utils import plot_model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input, Conv2D, Conv2DTranspose, MaxPooling2D, concatenate, Dropout
import tensorflow as tf

# Any results you write to the current directory are saved as output.
from IPython.display import display, Image

In [ ]:
(X_train,Y_train),(X_test, Y_test) = tf.keras.datasets.cifar10.load_data()


In [ ]:
X_train.shape

# Data Preprocessing

# noise factor: 0.07

In [ ]:
#NOrmalizing the images
x_train = X_train.astype('float32') / 255.
x_test = X_test.astype('float32') / 255.

#reshaping the images
x_train = np.reshape(x_train, (len(x_train), 32, 32, 3))  # adapt this if using `channels_first` image data format
x_test = np.reshape(x_test, (len(x_test), 32, 32, 3))  # adapt this if using `channels_first` image data format


#adding noise in data
noise_factor = 0.07

from skimage.util import random_noise

#noisy = random_noise(img, mode="poisson")
#just change the mode pf the noise to-->'gaussain', 
x_train_noisy = random_noise(x_train, mode="s&p",clip=True, amount=noise_factor)
x_test_noisy = random_noise(x_test, mode="s&p",clip=True, amount=noise_factor)

# Taking 100 images seperate for final testing

In [ ]:
x_test_final_noisy = x_test_noisy[-100:]
x_test_noisy = x_test_noisy[:-100]

x_test_final_original = x_test[-100:]
x_test = x_test[:-100]

# Visualization of 10 Data

In [ ]:
n = 10

In [ ]:
plt.figure(figsize=(20, 20))
for i in range(n):
    ax = plt.subplot(1, n, i+1)
    plt.imshow(x_train_noisy[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# One of the way we can achieve our goal of removing noise is AutoEncoder

**Copied from Keras Blog(https://blog.keras.io/building-autoencoders-in-keras.html):
**
* What are autoencoders good for?
* Today two interesting practical applications of autoencoders are data denoising (which we feature later in this post), and dimensionality reduction for data visualization. With appropriate dimensionality and sparsity constraints, autoencoders can learn data projections that are more interesting than PCA or other basic techniques.

**Refer to Keras Blog for better idea : https://blog.keras.io/building-autoencoders-in-keras.html**

## AutoEncoder Architecture

In [ ]:
display(Image(filename="/kaggle/input/images-architecture/images_architecture/unet.png"))

# Construction of Model

In [ ]:
def build_model(input_layer, start_neurons):
    conv1 = Conv2D(start_neurons*1,(3,3), activation='relu', padding='same')(input_layer)
    conv1 = Conv2D(start_neurons*1,(3,3), activation='relu', padding='same')(conv1)
    pool1 = MaxPooling2D((2,2))(conv1)
    pool1 = Dropout(0.25)(pool1)
    
    conv2 = Conv2D(start_neurons*2,(3,3), activation='relu', padding='same')(pool1)
    conv2 = Conv2D(start_neurons*2,(3,3), activation='relu', padding='same')(conv2)
    pool2 = MaxPooling2D((2,2))(conv2)
    pool2 = Dropout(0.5)(pool2)

    conv3 = Conv2D(start_neurons*4,(3,3), activation='relu', padding='same')(pool2)
    conv3 = Conv2D(start_neurons*4,(3,3), activation='relu', padding='same')(conv3)
    pool3 = MaxPooling2D((2,2))(conv3)
    pool3 = Dropout(0.5)(pool3)
    
    conv4 = Conv2D(start_neurons*8,(3,3), activation='relu', padding='same')(pool3)
    conv4 = Conv2D(start_neurons*8,(3,3), activation='relu', padding='same')(conv4)
    pool4 = MaxPooling2D((2,2))(conv4)
    pool4 = Dropout(0.5)(pool4)

    #Middle
    convm = Conv2D(start_neurons * 16, (3,3), activation='relu', padding='same')(pool4)
    convm = Conv2D(start_neurons * 16, (3,3), activation='relu', padding='same')(convm)
    
    #upconv part
    deconv4 = Conv2DTranspose(start_neurons*8,(3,3), strides=(2,2), padding='same')(convm)
    uconv4 = concatenate([deconv4, conv4])
    uconv4 = Dropout(0.5)(uconv4)
    uconv4 = Conv2D(start_neurons*8, (3,3), activation='relu', padding='same')(uconv4)
    uconv4 = Conv2D(start_neurons*8, (3,3), activation='relu', padding='same')(uconv4)
    
    deconv3 = Conv2DTranspose(start_neurons*8,(3,3), strides=(2,2), padding='same')(uconv4)
    uconv3 = concatenate([deconv3, conv3])
    uconv3 = Dropout(0.5)(uconv3)
    uconv3 = Conv2D(start_neurons*4, (3,3), activation='relu', padding='same')(uconv3)
    uconv3 = Conv2D(start_neurons*4, (3,3), activation='relu', padding='same')(uconv3)
    
    deconv2 = Conv2DTranspose(start_neurons*8,(3,3), strides=(2,2), padding='same')(uconv3)
    uconv2 = concatenate([deconv2, conv2])
    uconv2 = Dropout(0.5)(uconv2)
    uconv2 = Conv2D(start_neurons*2, (3,3), activation='relu', padding='same')(uconv2)
    uconv2 = Conv2D(start_neurons*2, (3,3), activation='relu', padding='same')(uconv2)
    
    deconv1 = Conv2DTranspose(start_neurons*8,(3,3), strides=(2,2), padding='same')(uconv2)
    uconv1 = concatenate([deconv1, conv1])
    uconv1 = Dropout(0.5)(uconv1)
    uconv1 = Conv2D(start_neurons*3, (3,3), activation='relu', padding='same')(uconv1)
    uconv1 = Conv2D(start_neurons*3, (3,3), activation='relu', padding='same')(uconv1)
    
    output_layer = Conv2D(3, (1,1), padding='same', activation='sigmoid')(uconv1)
    return output_layer

In [ ]:
#Defining target image size and Input size and building model
img_size_target = 32
input_layer = Input((img_size_target, img_size_target,3))
output_layer = build_model(input_layer,64)

In [ ]:
#Initializing and compiling model
model_unet = Model(input_layer, output_layer)
model_unet.compile(optimizer='adam', loss='MSE')

# AutoEncoder Summary

In [ ]:
model_unet.summary()

# Training Model

In [ ]:
model_unet.fit(x_train_noisy, x_train,
                epochs=3,
                batch_size=64,
                shuffle=True,
                validation_data=(x_test_noisy, x_test))

## Making Prediction

In [ ]:
predict = model_unet.predict(x_test_final_noisy)

# Visualizing the prediction

## Original Test images

In [ ]:
n=10

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(n):
    ax = plt.subplot(1, n, i+1)
    plt.imshow(x_test_final_original[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Noised Test images

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(n):
    ax = plt.subplot(1, n, i+1)
    plt.imshow(x_test_final_noisy[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Generated Test images

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(n):
    ax = plt.subplot(1, n, i+1)
    plt.imshow(predict[i])
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

# AutoEncoder: Train Loss VS validation loss

In [ ]:
model_unet.history.history

In [ ]:
epochs = range(len(model_unet.history.history['loss']))

plt.plot(epochs,model_unet.history.history['loss'],'r-o', label='Training Loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.figure()



In [ ]:
epochs = range(len(model_unet.history.history['loss']))

plt.plot(epochs,model_unet.history.history['loss'],'r-o', label='Training Loss')
plt.plot(epochs,model_unet.history.history['val_loss'],'b-o', label='Validation Loss')
plt.title('Training Loss vs Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.figure()



In [ ]:
from skimage.measure import compare_ssim, compare_psnr, compare_mse
from skimage import data, img_as_float

# MSE between Images

* Compare MSE between two images

In [ ]:
# Fourth Image
compare_mse(x_test_final_original[3], predict[3])

In [ ]:
# Fifth Image
compare_mse(x_test_final_original[4], predict[4])

In [ ]:
# Third Image
compare_mse(x_test_final_original[2], predict[2])

# For Whole final_test Dataset

In [ ]:
# whole dataset
compare_mse(x_test_final_original, predict)

# Structural Similarity Index

* When comparing images, the mean squared error (MSE)--while simple to implement--is not highly indicative of perceived similarity. Structural similarity aims to address this shortcoming by taking texture into account



In [ ]:
#fouth Image
compare_ssim(x_test_final_original[3], predict[3], multichannel=True)

In [ ]:
#fifth Image
compare_ssim(x_test_final_original[4], predict[4], multichannel=True)

In [ ]:
#Third Image
compare_ssim(x_test_final_original[2], predict[2], multichannel=True)

In [ ]:
# whole test dataset
compare_ssim(x_test_final_original, predict, multichannel=True)

# PSNR

Source: https://www.mathworks.com/help/vision/ref/psnr.html

* The PSNR block computes the peak signal-to-noise ratio, in decibels, between two images. This ratio is used as a quality measurement between the original and a compressed image. The higher the PSNR, the better the quality of the compressed, or reconstructed image.

* The mean-square error (MSE) and the peak signal-to-noise ratio (PSNR) are used to compare image compression quality. The MSE represents the cumulative squared error between the compressed and the original image, whereas PSNR represents a measure of the peak error. The lower the value of MSE, the lower the error.

In [ ]:
#fouth image
compare_psnr(x_test_final_original[3], predict[3])

In [ ]:
#fifth image
compare_psnr(x_test_final_original[4], predict[4])

In [ ]:
#third image
compare_psnr(x_test_final_original[2], predict[2])

In [ ]:
#whole final dataset
compare_psnr(x_test_final_original, predict)